### Predicting Price with Size

In [2]:
import warnings

import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import  LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.utils.validation import check_is_fitted


warnings.simplefilter(action="ignore", category=FutureWarning)


### Prepare Data

#### Import

In [4]:
df1=pd.read_csv("data/buenos-aires-real-estate-1.csv")

In [8]:
df1.head()
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4628 entries, 0 to 4627
Data columns (total 16 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   operation                   4628 non-null   object 
 1   property_type               4628 non-null   object 
 2   place_with_parent_names     4628 non-null   object 
 3   lat-lon                     4129 non-null   object 
 4   price                       4558 non-null   float64
 5   currency                    4558 non-null   object 
 6   price_aprox_local_currency  4558 non-null   float64
 7   price_aprox_usd             4558 non-null   float64
 8   surface_total_in_m2         1745 non-null   float64
 9   surface_covered_in_m2       4442 non-null   float64
 10  price_usd_per_m2            1247 non-null   float64
 11  price_per_m2                4271 non-null   float64
 12  floor                       302 non-null    float64
 13  rooms                       126 n